# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [34]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor

data_path = r'C:\Users\user\Desktop\GitHub\ML100-Days\alldata'
df = pd.read_csv(data_path + r'\titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head(10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [35]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
mean_Fare = df.groupby(['Sex'])['Fare'].mean().reset_index()
max_Fare = df.groupby(['Sex'])['Fare'].max().reset_index()
min_Fare = df.groupby(['Sex'])['Fare'].min().reset_index()
#Sex 對 Fare 取 mean, max, min
max_sibsp = df.groupby(['Cabin'])['SibSp'].max().reset_index()
max_C_F = df.groupby(['Cabin'])['Fare'].max().reset_index()
#Cabin 對 Sibsp 與 Fare 取max
temp = pd.merge(mean_Fare,max_Fare,how = 'left', on = ['Sex'])
temp = pd.merge(temp, min_Fare, how = 'left', on = ['Sex'])
temp.columns = ['Sex','mean_Fare','max_Fare','min_Fare']
temp2 = pd.merge(max_C_F,max_sibsp)
temp2.columns = ['Cabin','max_C_F','max_sibsp']
temp.head()
df = pd.merge(df,temp,how = 'left', on = ['Sex'])
df = pd.merge(df,temp2, how = 'left', on = ['Cabin'])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,mean_Fare,max_Fare,min_Fare,max_C_F,max_sibsp
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,25.523893,512.3292,0.00,NaN,NaN
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,44.479818,512.3292,6.75,71.2833,1.0
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,44.479818,512.3292,6.75,NaN,NaN
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,44.479818,512.3292,6.75,53.1000,1.0
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,25.523893,512.3292,0.00,NaN,NaN


In [36]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
obj_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
    elif dtype == 'object':
        obj_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')
print(f'{len(obj_features)} Numeric Features : {obj_features}\n')
# 削減文字型欄位, 只剩數值型欄位
df_obj = df[obj_features]
df = df[num_features]
df = df.fillna(0)
df_obj = df_obj.fillna('None')
MMEncoder = MinMaxScaler()
df_minus = df.drop(['mean_Fare','max_Fare','min_Fare','max_C_F','max_sibsp'],axis=1)
df.head()

#對 object 做 Label encoder
for c in obj_features:
    df_obj[c] = LabelEncoder().fit_transform(df_obj[c])
df_all = pd.concat([df_obj,df],axis =1)


10 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'mean_Fare', 'max_Fare', 'min_Fare', 'max_C_F', 'max_sibsp']

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?
#### ANS:有改善

In [38]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
MMscale = MinMaxScaler()
train_X = MMscale.fit_transform(df_minus)
estimator = LogisticRegression()
result = cross_val_score(estimator,train_X,train_Y,cv=5).mean()
print(f'Original data Score : {result}')

Original data Score : 0.707234411677194


D:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
D:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to sil

In [39]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMscale.fit_transform(df)
result = cross_val_score(estimator,train_X,train_Y,cv=5).mean()
print(f'Original+New data Score : {result}')

Original+New data Score : 0.7800428543006972


D:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
D:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to sil

In [40]:
#全特徵 + 邏輯回歸
train_X = MMscale.fit_transform(df_all)
result = cross_val_score(estimator,train_X,train_Y,cv=5).mean()
print(f'Original+New data Score : {result}')

Original+New data Score : 0.7834072928203442


D:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
D:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver

In [41]:
#利用OLS看其特徵對於結果的影響值
import statsmodels.api as sm
df_OLS = sm.add_constant(df_all)
train_X = df_OLS
OLS = sm.OLS(train_Y,train_X).fit()
print(OLS.summary())

                            OLS Regression Results                            
Dep. Variable:               Survived   R-squared:                       0.391
Model:                            OLS   Adj. R-squared:                  0.383
Method:                 Least Squares   F-statistic:                     46.98
Date:                Wed, 16 Jan 2019   Prob (F-statistic):           3.56e-86
Time:                        12:07:59   Log-Likelihood:                -400.99
No. Observations:                 891   AIC:                             828.0
Df Residuals:                     878   BIC:                             890.3
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Name       -7.107e-05   5.02e-05     -1.417      0.1

In [42]:
#刪去影響值低
df_modify = df_all.drop(['Parch','Cabin','max_C_F','Fare'],axis=1)

In [43]:
# 調整過後的資料 + 邏輯回歸
train_X = MMscale.fit_transform(df_modify)
result = cross_val_score(estimator,train_X,train_Y,cv=5).mean()
print(f'modify data Score : {result}')

modify data Score : 0.7923646715159691


D:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
D:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver